In [1]:
import sys
sys.path.append("../")
import os
import vqe_experiment as ve
import numpy as np

In [2]:
# define Hamiltonian: list of coefficients, list of Paulis, initial bitstring

# here: molecular Hamiltonian, parity mapping, two-qubit reduction
atom_string = "Li 0 0 0; H 0 0 2.0"              # PySCF atom string
num_orbitals = 3                                 # number of spatial orbitals
coeffs, paulis, HF_bitstring = ve.molecule(atom_string, new_num_orbitals=num_orbitals)

 /local/lmseifert/conda_envs/research/lib/python3.10/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning:PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.


In [3]:
coeffs, paulis, HF_bitstring

(array([-5.59396305e+00,  1.61387904e+00, -1.72701988e-02,  1.72701988e-02,
        -3.86183811e-02, -3.86183811e-02,  3.90387222e-01, -1.13923657e-02,
         1.13923657e-02,  2.52944297e-01,  1.61387904e+00, -1.72701988e-02,
         1.72701988e-02, -3.86183811e-02, -3.86183811e-02,  3.90387222e-01,
        -1.13923657e-02,  1.13923657e-02,  2.52944297e-01,  4.14787999e-01,
        -2.50295510e-02,  2.50295510e-02, -3.52792774e-02, -3.52792774e-02,
        -2.50295510e-02,  2.50295510e-02,  2.63398182e-03, -2.63398182e-03,
        -2.63398182e-03,  2.63398182e-03,  2.65122860e-03, -2.65122860e-03,
         2.65122860e-03, -2.65122860e-03, -3.52792774e-02, -3.52792774e-02,
         2.65122860e-03,  2.65122860e-03, -2.65122860e-03, -2.65122860e-03,
         5.49722197e-03,  5.49722197e-03,  5.49722197e-03,  5.49722197e-03,
         8.14827873e-02,  8.55526893e-04, -8.55526893e-04, -3.05064421e-03,
        -3.05064421e-03,  5.87484994e-03, -5.87484994e-03, -6.66607492e-04,
         6.6

In [4]:
# number of qubits
N = len(paulis[0])

In [5]:
# configure underlying VQE structure, passed as keyword argument dictionary

# a function which returns a parameterized Qiskit circuit (ansatz) and the number of parameters
ansatz_func = ve.efficientsu2_full
ansatz_reps = 1
# if True, the initial HF_bitstring will be appended after ansatz (layer of X gates where there are 1s)
# if False, HF_bitstring will be initialized before ansatz
init_last = False

vqe_kwargs = {
    "ansatz_func": ansatz_func,
    "ansatz_reps": ansatz_reps,
    "init_last": init_last,
    "HF_bitstring": HF_bitstring
}

In [6]:
ansatz, num_params = ansatz_func(N, ansatz_reps)
print(num_params)

16


In [7]:
ansatz.draw(fold=-1)

┌──────────┐┌──────────┐ ░                                ░  ┌──────────┐┌───────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├─░───■────■────■──────────────────░──┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐  │    │                  ░  ├──────────┤├───────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├─░─┤ X ├──┼────┼────■────■────────░──┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├
     ├──────────┤├──────────┤ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │        ░ ┌┴──────────┤├───────────┤
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─░──────┤ X ├──┼──┤ X ├──┼────■───░─┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├
     ├──────────┤├──────────┤ ░      └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░ ├───────────┤├───────────┤
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─░───────────┤ X ├─────┤ X ├┤ X ├─░─┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├
     └──────────┘└──────────┘ ░           └───┘     └───┘└───┘ ░ └───────────┘└───────────┘

CAFQA parameters are represented by a list $[x_0, x_1, x_2, ...]$ where $x_i$ parametrizes the $i$-th gate in the ansatz and can only take the discrete values $\{0,1,2,3\}$. Typically the parameterized gates are rotation gates, $R_y$ or $R_z$, so that $x_i$ corresponds to a rotation angle $x_i \cdot \pi/2$.

In [31]:
# configure algorithm

budget = 100                                                 # number of iterations
guess = list(map(int, np.random.randint(0, 4, num_params)))  # initial param guess
guess

[0, 3, 2, 2, 0, 3, 1, 0, 1, 1, 2, 2, 0, 2, 2, 2]

In [32]:
# get VQE circuit guess parameters (* pi/2)
qc = ve.vqe_circuit(
    N, 
    np.array(guess)*np.pi/2, 
    paulis, 
    ansatz_func=ansatz_func, 
    ansatz_reps=ansatz_reps, 
    init_last=init_last,
    HF_bitstring=HF_bitstring
    )

In [33]:
qc.draw(fold=-1)

┌───┐    ┌─────────┐ ┌─────────┐ ░                                ░ ┌─────────┐┌─────────┐
q3_0: ───┤ X ├────┤ Ry(0.0) ├─┤ Rz(0.0) ├─░───■────■────■──────────────────░─┤ Ry(π/2) ├┤ Rz(0.0) ├
      ┌──┴───┴───┐├─────────┴┐└─────────┘ ░ ┌─┴─┐  │    │                  ░ ├─────────┤└┬───────┬┘
q3_1: ┤ Ry(3π/2) ├┤ Rz(3π/2) ├────────────░─┤ X ├──┼────┼────■────■────────░─┤ Ry(π/2) ├─┤ Rz(π) ├─
      └──┬───┬───┘└┬───────┬─┘┌─────────┐ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │        ░ └┬───────┬┘ ├───────┤ 
q3_2: ───┤ X ├─────┤ Ry(π) ├──┤ Rz(π/2) ├─░──────┤ X ├──┼──┤ X ├──┼────■───░──┤ Ry(π) ├──┤ Rz(π) ├─
       ┌─┴───┴─┐  ┌┴───────┴┐ └─────────┘ ░      └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░  ├───────┤  ├───────┤ 
q3_3: ─┤ Ry(π) ├──┤ Rz(0.0) ├─────────────░───────────┤ X ├─────┤ X ├┤ X ├─░──┤ Ry(π) ├──┤ Rz(π) ├─
       └───────┘  └─────────┘             ░           └───┘     └───┘└───┘ ░  └───────┘  └───────┘ 
c3: 4/═════════════════════════════════════════════════════════════════════════════════════════════

In [34]:
# configure output

save_dir = "cafqa_example_out/"
os.system(f"mkdir -p {save_dir}")
# the output files are relative to save_dir
result_file = "result.txt"
loss_file = "cafqa_loss.txt"
params_file = "cafqa_params.txt"

In [35]:
# run CAFQA

# hypermapper changes the standard output to write to log files, so it needs to be
# changed back after CAFQA (otherwise printing breaks)
stdout = sys.stdout
cafqa_energy, cafqa_params = ve.run_cafqa(
    n_qubits=N,
    coeffs=coeffs,
    paulis=paulis,
    param_guess=list(guess),
    budget=budget,
    save_dir=save_dir,
    loss_file=loss_file,
    params_file=params_file,
    vqe_kwargs=vqe_kwargs
)
sys.stdout = stdout

Design of experiment phase, number of new doe samples = 100 .......
Loss computed by CAFQA VQE is -4.6133754287869175, in 0.012623327085748315 s.
Loss computed by CAFQA VQE is -4.384337868763828, in 0.013109657913446426 s.
Loss computed by CAFQA VQE is -6.369236960413943, in 0.011067484971135855 s.
Loss computed by CAFQA VQE is -4.99134903010928, in 0.013225100934505463 s.
Loss computed by CAFQA VQE is -5.092048368425278, in 0.011618433985859156 s.
Loss computed by CAFQA VQE is -4.844509088044406, in 0.01316904416307807 s.
Loss computed by CAFQA VQE is -3.180800676447161, in 0.01167025207541883 s.
Loss computed by CAFQA VQE is -4.683633835179146, in 0.01256012194789946 s.
Loss computed by CAFQA VQE is -4.55545786916967, in 0.01226162794046104 s.
Loss computed by CAFQA VQE is -4.998604668250714, in 0.01089441915974021 s.
Loss computed by CAFQA VQE is -1.1576511544917483, in 0.012176570016890764 s.
Loss computed by CAFQA VQE is -4.8475354816193255, in 0.012698953039944172 s.
Loss compute